# 教程：使用 Notebook 运行自动化分析流程

本 Notebook 用于分步执行 `main.py` 的核心逻辑。

In [ ]:
import yaml
from pathlib import Path
import sys
import os

current_dir = Path.cwd()

project_root = current_dir.parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src import get_papers, run_topic_modeling, analyze

print(f"✅ Environment setup complete. Project root set to: {project_root}")

## 步骤二：指定配置文件

在此单元格中，指定要运行的配置文件路径和是否强制重跑，以模拟命令行输入。

In [ ]:
# --- User Configuration Area ---

# 1. Specify the path to your configuration file
config_path = "../configs/iclr_2025_full_analysis.yaml"

# 2. (Optional) Set to True to force rerun all steps
force_rerun = False

# ------------------------------------

# Simulate argparse
class Args:
    def __init__(self, config, force_rerun):
        self.config = config
        self.force_rerun = force_rerun

args = Args(config_path, force_rerun)

print(f"Configuration file to be used: {args.config}")
print(f"Force rerun enabled: {args.force_rerun}")

if not Path(args.config).exists():
    print(f"❌ Error: Configuration file '{args.config}' not found. Please check the path.")

## 步骤三：加载配置并准备目录

In [ ]:
def load_config(config_path: str) -> dict:
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    return config

def get_expected_filepaths(config: dict, raw_dir: Path, processed_dir: Path) -> dict:
    conf_id = config['conference_id'].replace('/', '_').replace('.', '')
    limit = config.get('limit', None)
    fetch_reviews = config.get('fetch_reviews', False)

    suffix = "_reviews" if fetch_reviews else ""
    limit_suffix = f"_limit{limit}" if limit else ""
    raw_filename = f"{conf_id}_papers{suffix}{limit_suffix}.jsonl"
    raw_path = raw_dir / raw_filename

    processed_filename = f"{raw_path.stem}_with_topics.csv"
    processed_path = processed_dir / processed_filename
    
    return {"raw": raw_path, "processed": processed_path}

print(f"📖 Loading configuration from: {args.config}")
config = load_config(args.config)

output_folder_name = config.get('output_folder_name', 'default_analysis')
output_dir = Path("results") / output_folder_name
data_raw_dir = Path("data/raw")
data_processed_dir = Path("data/processed")

output_dir.mkdir(parents=True, exist_ok=True)
print(f"📂 Results will be saved to: {output_dir}")

expected_paths = get_expected_filepaths(config, data_raw_dir, data_processed_dir)
raw_data_path = expected_paths['raw']
processed_data_path = expected_paths['processed']

## 步骤四：数据获取

In [ ]:
print("\n--- [Step 1/3] Data Fetching ---")
if not args.force_rerun and raw_data_path.exists():
    print(f"✅ File already exists, skipping data fetching.")
    print(f"   --> Using local file: {raw_data_path}")
    data_fetch_success = True
else:
    print("   --> Starting data fetching (local file not found or --force-rerun)...")
    returned_path = get_papers.main(config=config, raw_data_dir=data_raw_dir)
    if not returned_path:
        print("❌ Data fetching failed. Terminating process.")
        data_fetch_success = False
    else:
        data_fetch_success = True
        print("--- [Step 1/3] Data Fetching Complete ---\n")

if not data_fetch_success:
    # Stop execution if this step failed
    raise SystemExit("Stopping notebook execution due to data fetching failure.")

## 步骤五：主题建模

In [ ]:
if config.get('topic_modeling', {}).get('enabled', False):
    print("--- [Step 2/3] Topic Modeling ---")
    if not args.force_rerun and processed_data_path.exists():
        print(f"✅ File already exists, skipping topic modeling.")
        print(f"   --> Using local file: {processed_data_path}")
    else:
        print("   --> Starting topic modeling (local file not found or --force-rerun)...")
        if not raw_data_path.exists():
             print(f"❌ Input file {raw_data_path} does not exist. Cannot perform topic modeling.")
        else:
            returned_path = run_topic_modeling.main(
                config=config, input_path=raw_data_path, 
                processed_data_dir=data_processed_dir, output_dir=output_dir
            )
            if not returned_path:
                print("❌ Topic modeling failed. Terminating process.")
                raise SystemExit("Stopping notebook execution due to topic modeling failure.")
            else:
                print("--- [Step 2/3] Topic Modeling Complete ---\n")
else:
    print("--- [Step 2/3] 🟢 Topic modeling is disabled in the configuration. ---")

## 步骤六：分析与可视化

In [ ]:
if config.get('analysis', {}).get('enabled', False):
    print("--- [Step 3/3] Analysis and Visualization ---")
    if processed_data_path.exists():
        analyze.main(config=config, input_path=processed_data_path, output_dir=output_dir)
    else:
        print(f"⚠️ Cannot perform analysis because the input file {processed_data_path} does not exist.")
    print("--- [Step 3/3] Analysis and Visualization Complete ---\n")
else:
    print("--- [Step 3/3] 🟢 Analysis and visualization is disabled in the configuration. ---")

print("🎉🎉🎉 All processes completed successfully! 🎉🎉🎉")